In [24]:
import pandas as pd
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [25]:
# Loading the csv file with the info about the wav files
metadata = pd.read_csv('/Users/adi/Downloads/archive (17)/bird_songs_metadata.csv')
metadata.head()

,id,genus,species,subspecies,name,recordist,country,location,latitude,longitude,altitude,sound_type,source_url,license,time,date,remarks,filename
0,557838,Thryomanes,bewickii,NaN,Bewick's Wren,Whitney Neufeld-Kaiser,United States,"Arlington, Snohomish County, Washington",48.0708,-122.1006,100,"adult, sex uncertain, song",//www.xeno-canto.org/557838,//creativecommons.org/licenses/by-nc-sa/4.0/,11:51,2020-03-14,"Recorded with Voice Record Pro on iPhone7, nor...",557838-0.wav
1,557838,Thryomanes,bewickii,NaN,Bewick's Wren,Whitney Neufeld-Kaiser,United States,"Arlington, Snohomish County, Washington",48.0708,-122.1006,100,"adult, sex uncertain, song",//www.xeno-canto.org/557838,//creativecommons.org/licenses/by-nc-sa/4.0/,11:51,2020-03-14,"Recorded with Voice Record Pro on iPhone7, nor...",557838-1.wav
2,557838,Thryomanes,bewickii,NaN,Bewick's Wren,Whitney Neufeld-Kaiser,United States,"Arlington, Snohomish County, Washington",48.0708,-122.1006,100,"adult, sex uncertain, song",//www.xeno-canto.org/557838,//creativecommons.org/licenses/by-nc-sa/4.0/,11:51,2020-03-14,"Recorded with Voice Record Pro on iPhone7, nor...",557838-4.wav
3,557838,Thryomanes,bewickii,NaN,Bewick's Wren,Whitney Neufeld-Kaiser,United States,"Arlington, Snohomish County, Washington",48.0708,-122.1006,100,"adult, sex uncertain, song",//www.xeno-canto.org/557838,//creativecommons.org/licenses/by-nc-sa/4.0/,11:51,2020-03-14,"Recorded with Voice Record Pro on iPhone7, nor...",557838-5.wav
4,557838,Thryomanes,bewickii,NaN,Bewick's Wren,Whitney Neufeld-Kaiser,United States,"Arlington, Snohomish County, Washington",48.0708,-122.1006,100,"adult, sex uncertain, song",//www.xeno-canto.org/557838,//creativecommons.org/licenses/by-nc-sa/4.0/,11:51,2020-03-14,"Recorded with Voice Record Pro on iPhone7, nor...",557838-6.wav


In [26]:
# Lists for storing the filepaths and labels
file_paths = []
labels = []

audio_dir = '/Users/adi/Downloads/archive (17)/wavfiles/'

for index, row in metadata.iterrows():
    file_path = os.path.join(audio_dir, row['filename'])
    if os.path.exists(file_path):
        file_paths.append(file_path)
        labels.append(row['species'])

file_paths[:5], labels[:5]

(['/Users/adi/Downloads/archive (17)/wavfiles/557838-0.wav',
  '/Users/adi/Downloads/archive (17)/wavfiles/557838-1.wav',
  '/Users/adi/Downloads/archive (17)/wavfiles/557838-4.wav',
  '/Users/adi/Downloads/archive (17)/wavfiles/557838-5.wav',
  '/Users/adi/Downloads/archive (17)/wavfiles/557838-6.wav'],
 ['bewickii', 'bewickii', 'bewickii', 'bewickii', 'bewickii'])

In [29]:
# Function to extract Mel Spectrogram from wav files
def extract_log_mel_spectrogram_features(file_paths):
    log_mel_spectrogram_features = []
    for file_path in file_paths:
        y, sr = librosa.load(file_path)
        mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
        log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
        # Flatten spectrogram to 1D 
        log_mel_spectrogram_flat = log_mel_spectrogram.flatten()
        log_mel_spectrogram_features.append(log_mel_spectrogram_flat)
    return np.array(log_mel_spectrogram_features)

X = extract_log_mel_spectrogram_features(file_paths)
X[:5]

array([[-22.416008 , -17.302883 ,  -9.898376 , ..., -77.59977  ,
        -67.687805 , -59.199257 ],
       [-16.61066  , -13.014887 , -13.764468 , ..., -78.35228  ,
        -76.9274   , -72.548996 ],
       [-19.372046 , -15.390911 , -13.366266 , ..., -75.46674  ,
        -70.00602  , -62.78939  ],
       [-11.709303 , -11.232471 ,  -6.2762127, ..., -74.10058  ,
        -75.8924   , -76.88263  ],
       [-13.154354 ,  -9.964792 , -12.25135  , ..., -72.28287  ,
        -70.355896 , -65.24282  ]], dtype=float32)

In [30]:
# Converting labels to numeric
label_dict = {label: index for index, label in enumerate(set(labels))}
y = np.array([label_dict[label] for label in labels])
y[:5]

array([1, 1, 1, 1, 1])

In [31]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
# RandomForest Classifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [33]:
# Predict and Calculate Accuracy
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.7179723502304147